Source 

https://data.world/vizzup/mental-health-depression-disorder-data
https://datahub.io/core/country-list

About 
The dataset contains informative data from countries across the globe about the prevalence of mental health disorders including schizophrenia, bipolar disorder, eating disorders, anxiety disorders, drug use disorders, depression and alcohol use disorders.

Questions to thinks about 
Is depression linked with suicide and what is the percentage rate?

How many people in each country suffer mental health problems?


In [1]:
#import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import os, zipfile

In [2]:
#Read in Mental health disorder data 
data_path = '/Users/yeboahi/MSDS/GIS Data/import_data/Mental health Depression disorder Data.csv'
#Read in country coordinates 
geo_coord = '/Users/yeboahi/MSDS/GIS Data/import_data/countries 2.csv'
#Read in shape file
shape_path = '/Users/yeboahi/MSDS/GIS Data/import_data/USACANAB.SHP'

geo_mh = pd.read_csv(data_path)
geo_coord = pd.read_csv(geo_coord)

#let's rename the columns to something shorter
map_names = {'Entity': 'country',
             'Code': 'code',
             'Year' : 'year',
             'Schizophrenia (%)':'schizo', 
             'Bipolar disorder (%)':'bipolar',
             'Eating disorders (%)' : 'eating dis',
             'Anxiety disorders (%)' : 'anxiety',
             'Drug use disorders (%)':'drug',
             'Depression (%)':'depres',
             'Alcohol use disorders (%)':'alcohol'}

geo_mh.rename(columns=map_names,inplace=True)

geo_mh['schizo'] = pd.to_numeric(geo_mh['schizo'], errors='coerce')
geo_mh['bipolar'] = pd.to_numeric(geo_mh['bipolar'], errors='coerce')
geo_mh['eating dis'] = pd.to_numeric(geo_mh['eating dis'], errors='coerce')



geo_mh = geo_mh.drop(columns= ['index'])
print(geo_mh.info())


/var/folders/qg/d9jygk1d6dv3c6s4t5ymzpgr0000gn/T/ipykernel_9801/631915699.py:8: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  geo_mh = pd.read_csv(data_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108553 entries, 0 to 108552
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   country     108553 non-null  object 
 1   code        103141 non-null  object 
 2   year        108553 non-null  object 
 3   schizo      25872 non-null   float64
 4   bipolar     19404 non-null   float64
 5   eating dis  100234 non-null  float64
 6   anxiety     6468 non-null    float64
 7   drug        6468 non-null    float64
 8   depres      6468 non-null    float64
 9   alcohol     6468 non-null    float64
dtypes: float64(7), object(3)
memory usage: 8.3+ MB
None


In [3]:
#Dropping the rows which have non-numerical data, and so we concentrate on the rows up to 6467 for now
#geo_mh.dropna(how='all')


In [4]:
print(geo_coord.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   country    244 non-null    object 
 1   latitude   244 non-null    float64
 2   longitude  244 non-null    float64
 3   name       245 non-null    object 
dtypes: float64(2), object(2)
memory usage: 7.8+ KB
None


In [5]:
geo_mh.head()

,country,code,year,schizo,bipolar,eating dis,anxiety,drug,depres,alcohol
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [6]:
#Using lat and long to get geo points
geo_coord = gpd.GeoDataFrame(geo_coord, geometry=gpd.points_from_xy(geo_coord.latitude, geo_coord.longitude))


In [7]:
geo_coord.head()

,country,latitude,longitude,name,geometry
0,AD,42.546245,1.601554,Andorra,POINT (42.54624 1.60155)
1,AE,23.424076,53.847818,United Arab Emirates,POINT (23.42408 53.84782)
2,AF,33.939110,67.709953,Afghanistan,POINT (33.93911 67.70995)
3,AG,17.060816,-61.796428,Antigua and Barbuda,POINT (17.06082 -61.79643)
4,AI,18.220554,-63.068615,Anguilla,POINT (18.22055 -63.06862)


In [8]:
geo_mh.rename(columns = {'country':'name'}, inplace = True)


In [9]:
geo_mh.head()

,name,code,year,schizo,bipolar,eating dis,anxiety,drug,depres,alcohol
0,Afghanistan,AFG,1990,0.160560,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [10]:
geo_mh = geo_mh.merge(geo_coord, on='name')

In [11]:
geo_mh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95924 entries, 0 to 95923
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   name        95924 non-null  object  
 1   code        95924 non-null  object  
 2   year        95924 non-null  object  
 3   schizo      20608 non-null  float64 
 4   bipolar     15456 non-null  float64 
 5   eating dis  90772 non-null  float64 
 6   anxiety     5152 non-null   float64 
 7   drug        5152 non-null   float64 
 8   depres      5152 non-null   float64 
 9   alcohol     5152 non-null   float64 
 10  country     95428 non-null  object  
 11  latitude    95924 non-null  float64 
 12  longitude   95924 non-null  float64 
 13  geometry    95924 non-null  geometry
dtypes: float64(9), geometry(1), object(4)
memory usage: 11.0+ MB


In [12]:
#Have a quick look at the data through the years for one country
usa = geo_mh[geo_mh['code'] == 'USA']
#usa.dropna(how='all')
print(usa)

                name code  year        schizo   bipolar  eating dis   anxiety  \
85000  United States  USA  1990  3.408970e-01  0.653706    0.468676  6.576884   
85001  United States  USA  1991  3.389130e-01  0.653953    0.473575  6.553351   
85002  United States  USA  1992  3.373430e-01  0.654205    0.478610  6.536143   
85003  United States  USA  1993  3.362070e-01  0.654431    0.483539  6.525536   
85004  United States  USA  1994  3.355360e-01  0.654620    0.488235  6.522278   
...              ...  ...   ...           ...       ...         ...       ...   
85491  United States  USA  2013  1.485464e+07       NaN         NaN       NaN   
85492  United States  USA  2014  1.500015e+07       NaN         NaN       NaN   
85493  United States  USA  2015  1.515513e+07       NaN         NaN       NaN   
85494  United States  USA  2016  1.532168e+07       NaN         NaN       NaN   
85495  United States  USA  2017  1.549936e+07       NaN         NaN       NaN   

           drug    depres  

In [13]:
# create list of countries for North America to extract these into a separate dataframe
country_na = ['United States','Mexico',
                  'Greenland','Canada',
                  'Cuba','Jamaica',
                  'Panama','Haiti',
                  'Puerto Rico','Costa Rica',
                  'Dominican Republic','Guatemala',
                  'Honduras','Belize',
                  'The Bahamas...','Nicaragua',
                  'Martinique','Sint Maarten','Turks and Caicos Islands',
                  'Grenada','Saint Kitts and Nevis','Cayman Islands',
                  'United States Minor Outlying Islands','Antigua Barbuda','Anguilla',
                  'Dominica','Saint Lucia','Saint Vincet and the Grenadines',
                  'British Virgins Islands','Saint Pierre and Miquelon','United States Virgin Islands',
                  'Saint Barthelemy','Momtserrat','Saint Martin'] 

In [14]:
usa = geo_mh[geo_mh['name'].isin(country_na)]
#print(usa)
usa.head()

,name,code,year,schizo,bipolar,eating dis,anxiety,drug,depres,alcohol,country,latitude,longitude,geometry
8628,Belize,BLZ,1990,0.190381,0.870972,0.211610,4.489253,0.809323,2.717497,1.542925,BZ,17.189877,-88.49765,POINT (17.18988 -88.49765)
8629,Belize,BLZ,1991,0.191191,0.871933,0.216008,4.494375,0.811845,2.725553,1.572840,BZ,17.189877,-88.49765,POINT (17.18988 -88.49765)
8630,Belize,BLZ,1992,0.191919,0.872827,0.220258,4.499428,0.814605,2.735803,1.600760,BZ,17.189877,-88.49765,POINT (17.18988 -88.49765)
8631,Belize,BLZ,1993,0.192536,0.873649,0.223665,4.504220,0.815405,2.745329,1.626030,BZ,17.189877,-88.49765,POINT (17.18988 -88.49765)
8632,Belize,BLZ,1994,0.193004,0.874343,0.226725,4.508290,0.816487,2.755676,1.647890,BZ,17.189877,-88.49765,POINT (17.18988 -88.49765)


In [15]:
shape = gpd.read_file(shape_path)
shape.info()
#shape.plot('geometry')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2084 entries, 0 to 2083
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   AREA        2084 non-null   float64 
 1   PERIMETER   2084 non-null   float64 
 2   USACANAB_   2084 non-null   int64   
 3   USACANAB_I  2084 non-null   int64   
 4   STATE       2083 non-null   object  
 5   geometry    2084 non-null   geometry
dtypes: float64(2), geometry(1), int64(2), object(1)
memory usage: 97.8+ KB


In [16]:
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 


/Users/yeboahi/anaconda3/lib/python3.11/site-packages/google/colab/data_table.py:30: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



In [17]:
# Alcohol Abuse
alc_data = dict(
        type = 'choropleth',
        colorscale = 'turbo',
        reversescale = True,
        locations = usa['name'],
        locationmode = "country names",
        z = usa['alcohol'],
        text = usa['country'],
        colorbar = {'title' : 'Alcohol Abuse %'},
      ) 
alc_layout = dict(title = 'Prevalence of Alcohol Abuse in North America in 2017', 
              geo = dict({'scope':'north america'}, projection = {'type':'natural earth'}, showframe = True))

In [18]:
go.Figure(data = [alc_data],layout = alc_layout) 
choromap = go.Figure(data = [alc_data],layout = alc_layout) 
iplot(choromap)



In [101]:
# Depression
dep_data = dict(
        type = 'choropleth',
        colorscale = 'Viridis',
        reversescale = True,
        locations = usa['name'],
        locationmode = "country names",
        z = usa['depres'],
        text = usa['country'],
        colorbar = {'title' : 'Depression %'},
      ) 
dep_layout = dict(title = 'Prevalence of Depressive Disorder in North America in 2017', 
              geo = dict({'scope':'north america'}, projection = {'type':'natural earth'}, showframe = True))

In [102]:
go.Figure(data = [dep_data],layout = dep_layout) 
choromap = go.Figure(data = [dep_data],layout = dep_layout) 
iplot(choromap)